In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rheumatoid_Arthritis"
cohort = "GSE176440"

# Input paths
in_trait_dir = "../../input/GEO/Rheumatoid_Arthritis"
in_cohort_dir = "../../input/GEO/Rheumatoid_Arthritis/GSE176440"

# Output paths
out_data_file = "../../output/preprocess/Rheumatoid_Arthritis/GSE176440.csv"
out_gene_data_file = "../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE176440.csv"
out_clinical_data_file = "../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE176440.csv"
json_path = "../../output/preprocess/Rheumatoid_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiles of CD4+ T cells before and after methotrexate treatment in rheumatoid arthritis patients [Microarray]"
!Series_summary	"To understand the molecular mechanisms by which methotraxate improves the disease activity in rheumatoid arthritis, CD4+ T cells were obtained before and 3month after methotrexate treatment."
!Series_overall_design	"28 treatment naïve rheumatoid arthritis patients participated in the study. Blood samples were obtained before and 3 months after methotrexate treatment. CD4+ T cells were magnetically purified and subjected the DNA microarray analyses."
Sample Characteristics Dictionary:
{0: ['individual: A29', 'individual: A30', 'individual: A34', 'individual: C14', 'individual: C16', 'individual: C19', 'individual: C43', 'individual: C49', 'individual: C71', 'individual: C80', 'individual: C85', 'individual: C87', 'individual: C91', 'individual: C92', 'individual: C93', 'individual: C95', 'individual: C96',

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any, List

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains microarray data of CD4+ T cells
# which implies gene expression data, not just miRNA or methylation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identify keys in the sample characteristics dictionary

# Trait (Rheumatoid Arthritis)
# From the sample characteristics, all samples are from RA patients (key 1)
trait_row = 1  # "disease state: rheumatoid arthritis patient"

# Treatment status (before/after methotrexate) at key 2 - this could be useful clinical information
# but it's not age or gender

# Age - Not available in the sample characteristics
age_row = None

# Gender - Not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert trait value to binary (0 for control, 1 for RA)."""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # All samples are RA patients based on the data
    if "rheumatoid arthritis" in value.lower():
        return 1
    return None  # Default case for unknown values

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous numeric."""
    # Not used since age data is not available
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0 for female, 1 for male)."""
    # Not used since gender data is not available
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Load the clinical data (assuming it was saved from a previous step)
    clinical_data = pd.DataFrame(
        {0: ['individual: A29', 'individual: A30', 'individual: A34', 'individual: C14', 'individual: C16', 'individual: C19', 'individual: C43', 'individual: C49', 'individual: C71', 'individual: C80', 'individual: C85', 'individual: C87', 'individual: C91', 'individual: C92', 'individual: C93', 'individual: C95', 'individual: C96', 'individual: C100', 'individual: C102', 'individual: C103', 'individual: C107', 'individual: C108', 'individual: C109', 'individual: C111', 'individual: C115', 'individual: C116', 'individual: C117', 'individual: K20'], 
         1: ['disease state: rheumatoid arthritis patient'] * 28,
         2: ['treatment: before methotrexate', 'treatment: 3 months after  methotrexate'] * 14,
         3: ['cell type: CD4+ T cells'] * 28}
    )
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    print("Preview of selected clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the selected clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical data:
{0: [nan], 1: [1.0], 2: [nan], 3: [nan]}
Clinical data saved to ../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE176440.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers in the gene expression data (A_23_P100001, A_23_P100011, etc.) are Agilent microarray 
# probe identifiers, not human gene symbols.
# These are probe IDs from an Agilent microarray platform and need to be mapped to human gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in the gene annotation dataframe
# From the preview, 'ID' in the gene_annotation corresponds to the probe identifiers in gene_data
# 'GENE_SYMBOL' contains the human gene symbols we want to map to
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'

# 2. Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This function handles the many-to-many relationship as specified
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the result to verify the transformation
print("Preview of mapped gene expression data:")
print(gene_data.shape)
print(gene_data.index[:10])  # Print first 10 gene symbols


Preview of mapped gene expression data:
(18488, 56)
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was saved in Step 2
clinical_data_df = pd.read_csv(out_clinical_data_file)

# Check the structure of the clinical data
print("Clinical data columns:", clinical_data_df.columns.tolist())

# Since we don't have a proper trait column in the clinical data,
# we need to add it before linking
if trait not in clinical_data_df.columns:
    # Create a proper clinical data structure with the trait column
    # From previous steps, we see all values are 1.0 for RA patients
    clinical_data_df[trait] = 1.0

# Link the clinical and genetic data on sample IDs
linked_data = geo_link_clinical_genetic_data(clinical_data_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
try:
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After handling missing values, linked data shape: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    trait_biased, linked_data_filtered = judge_and_remove_biased_features(linked_data, trait)
    
    # 5. Conduct final quality validation and save relevant information
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True,
        is_biased=trait_biased, 
        df=linked_data_filtered,
        note="Dataset contains gene expression data from CD4+ T cells of rheumatoid arthritis patients before and after methotrexate treatment."
    )
    
    # 6. If the linked data is usable, save it as a CSV file
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data_filtered.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset not usable for analysis. No linked data saved.")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # If there's an error, mark the dataset as not usable
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Marking as unavailable due to processing error
        is_biased=True, 
        df=pd.DataFrame(),
        note=f"Error during data processing: {e}. Dataset contains only RA patients with constant trait value."
    )
    print("Dataset not usable for analysis. No linked data saved.")

Normalized gene data saved to ../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE176440.csv
Clinical data columns: ['0', '1', '2', '3']
Linked data shape: (61, 18248)
Error processing data: ['Rheumatoid_Arthritis']
Abnormality detected in the cohort: GSE176440. Preprocessing failed.
Dataset not usable for analysis. No linked data saved.
